In [3]:
import pandas as pd


csv_url = 'https://raw.githubusercontent.com/jgehrcke/covid-19-germany-gae/master/cases-rki-by-ags.csv'

df = pd.read_csv(csv_url, sep=',', index_col=0, parse_dates=True) # it's important to define index and parse_dates


In [2]:
pd.set_option('display.max_columns', None)
df.tail(10)

,1001,1002,1003,1004,1051,1053,1054,1055,1056,1057,1058,1059,1060,1061,1062,2000,3101,3102,3103,3151,3153,3154,3155,3157,3158,3159,3241,3251,3252,3254,3255,3256,3257,3351,3352,3353,3354,3355,3356,3357,3358,3359,3360,3361,3401,3402,3403,3404,3405,3451,3452,3453,3454,3455,3456,3457,3458,3459,3460,3461,3462,4011,4012,5111,5112,5113,5114,5116,5117,5119,5120,5122,5124,5154,5158,5162,5166,5170,5314,5315,5316,5334,5358,5362,5366,5370,5374,5378,5382,5512,5513,5515,5554,5558,5562,5566,5570,5711,5754,5758,5762,5766,5770,5774,5911,5913,5914,5915,5916,5954,5958,5962,5966,5970,5974,5978,6411,6412,6413,6414,6431,6432,6433,6434,6435,6436,6437,6438,6439,6440,6531,6532,6533,6534,6535,6611,6631,6632,6633,6634,6635,6636,7111,7131,7132,7133,7134,7135,7137,7138,7140,7141,7143,7211,7231,7232,7233,7235,7311,7312,7313,7314,7315,7316,7317,7318,7319,7320,7331,7332,7333,7334,7335,7336,7337,7338,7339,7340,8111,8115,8116,8117,8118,8119,8121,8125,8126,8127,8128,8135,8136,8211,8212,8215,8216,8221,8222,8225,8226,8231,8235,8236,8237,8311,8315,8316,8317,8325,8326,8327,8335,8336,8337,8415,8416,8417,8421,8425,8426,8435,8436,8437,9161,9162,9163,9171,9172,9173,9174,9175,9176,9177,9178,9179,9180,9181,9182,9183,9184,9185,9186,9187,9188,9189,9190,9261,9262,9263,9271,9272,9273,9274,9275,9276,9277,9278,9279,9361,9362,9363,9371,9372,9373,9374,9375,9376,9377,9461,9462,9463,9464,9471,9472,9473,9474,9475,9476,9477,9478,9479,9561,9562,9563,9564,9565,9571,9572,9573,9574,9575,9576,9577,9661,9662,9663,9671,9672,9673,9674,9675,9676,9677,9678,9679,9761,9762,9763,9764,9771,9772,9773,9774,9775,9776,9777,9778,9779,9780,10041,10042,10043,10044,10045,10046,11001,11002,11003,11004,11005,11006,11007,11008,11009,11010,11011,11012,12051,12052,12053,12054,12060,12061,12062,12063,12064,12065,12066,12067,12068,12069,12070,12071,12072,12073,13003,13004,13071,13072,13073,13074,13075,13076,14511,14521,14522,14523,14524,14612,14625,14626,14627,14628,14713,14729,14730,15001,15002,15003,15081,15082,15083,15084,15085,15086,15087,15088,15089,15090,15091,16051,16052,16053,16054,16055,16056,16061,16062,16063,16064,16065,16066,16067,16068,16069,16070,16071,16072,16073,16074,16075,16076,16077,11000,sum_cases
time_iso8601,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2021-03-16 17:00:00+00:00,1803,3646,4475,1259,1384,3432,2186,2323,7845,956,2882,2239,4739,1465,5116,55917,4165,2993,2204,3952,1784,1674,1616,3595,1824,5719,34438,4770,3070,6485,1129,2787,3103,2932,3317,4750,524,2155,1966,2628,2341,3303,1556,2768,2401,633,2777,5117,1334,2113,2325,7845,8464,1184,3856,2625,3535,11708,6091,2317,863,16382,2770,17927,20384,18846,7665,7789,5821,7858,3946,6680,13668,7551,16478,12283,8414,12388,9862,36045,5079,19146,9941,13004,5355,8681,9226,7831,16475,4029,10667,5859,9659,4053,23065,11199,8703,10380,14278,7886,4014,11898,10765,6842,11198,19284,8082,7189,6825,9596,6492,13175,4010,7469,5952,13465,4203,27901,6130,9166,7768,8068,10023,5766,15942,6263,3267,13337,5255,9634,9188,8037,6134,7364,2672,5312,8743,3533,5474,4709,4419,2407,3214,3238,3007,4314,2327,1568,4686,5509,2687,2860,4781,1953,2159,2444,1474,3294,1592,2492,1155,7140,6977,1264,944,2048,2663,504,3459,3248,1674,3891,2789,1628,2590,4874,5340,1894,19632,11781,17648,7817,17868,13284,5829,10347,3416,6374,3484,3443,9782,1350,7208,12704,6246,4017,11458,4428,16010,5422,5947,6578,3554,5626,6578,4650,13397,5300,6380,5018,7801,7691,5315,9451,6329,5687,3707,5922,5266,5202,7112,3368,4221,56131,3106,4243,4645,3657,6016,4942,3459,5335,5993,7381,2641,3099,3321,4907,11779,2875,3983,11185,3903,7907,3483,2874,2875,1631,4661,3779,3450,5379,10597,4459,5450,3732,3561,1133,4735,2305,3464,4494,4370,4385,5315,5789,4689,226